In [0]:
from IPython.display import clear_output

In [0]:
!pip install git+https://github.com/lopuhin/python-adagram.git
!pip install pycodestyle flake8 pycodestyle_magic
!pip install wiki-ru-wordnet
!pip install pymorphy2[fast]
clear_output()

In [0]:
%load_ext pycodestyle_magic

In [0]:
import re
import adagram
import warnings
import pandas as pd
import numpy as np
from pprint import pprint
from collections import Counter
from pymorphy2 import MorphAnalyzer
from wiki_ru_wordnet import WikiWordnet
from nltk.tokenize import RegexpTokenizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [0]:
warnings.filterwarnings('ignore')

#**Подбор контекстов**
Для слова *острый* я выбрала следующие значения из МАС:

*   Имеющий хорошо колющий конец или хорошо режущий край; противоп. тупой.
*   Сильно действующий на органы чувств (обоняние, вкус, слух, зрение); резкий.
*   Крайне напряженный; критический.<br>

Контексты для каждого из значений:

*   *острый* + инструмент или оружие
*   *острый* + запах или вкус
*   *острый* + взаимодействие или взаимоотношение

In [0]:
!wget https://raw.githubusercontent.com/vyhuholl/hw_compling_4th_year/master/ruscorpora_sharp_objects.csv
!wget https://raw.githubusercontent.com/vyhuholl/hw_compling_4th_year/master/ruscorpora_sharp_smell.csv
!wget https://raw.githubusercontent.com/vyhuholl/hw_compling_4th_year/master/ruscorpora_sharp_conflict.csv
clear_output()

In [0]:
df_sharp_objects = pd.read_csv('ruscorpora_sharp_objects.csv')

In [8]:
df_sharp_objects.head()

,Reversed left context,Reversed center,Left context,Center,Punct,Right context,Title,Author,Birthday,Header,Created,Sphere,Type,Topic,Publication,Publ_year,Medium,Ambiguity,Full context
0,"етинкторп ,вотог ьсуг отч ,ясьтидебу",мыртсо,"убедиться, что гусь готов, проткните",острым,,ножом бедро.,Рецепты национальных кухонь: Скандинавская кух...,NaN,NaN,Рецепты национальных кухонь: Скандинавская кухня,2000-2005,"бытовая, нехудожественная, производственно-тех...",рецепт,дом и домашнее хозяйство,NaN,NaN,электронный текст,омонимия снята,"Для того, чтобы убедиться, что гусь готов, про..."
1,йошьлоб и ― иворк в,йыртсо,в крови ― и большой,острый,,нож.,Запись LiveJournal (2004),NaN,NaN,Запись LiveJournal,2004,"нехудожественная, электронная коммуникация",блог,частная жизнь,NaN,NaN,электронный текст,омонимия снята,"В руках у безумца ― портфель, в котором лежат ..."
2,тюазерорп и укйенил тюавыдалкирп метаЗ,мыртсо,Затем прикладывают линейку и прорезают,острым,,ножом уложенные внахлест оба полотнища.,Елена Волкова. Тот самый линолеум // «Биржа пл...,Елена Волкова,NaN,Тот самый линолеум,2002,"нехудожественная, публицистика",статья,дом и домашнее хозяйство,«Биржа плюс свой дом» (Н. Новгород),2002.08.12,газета,омонимия снята,Затем прикладывают линейку и прорезают острым ...
3,"хынназероп ,хывенчирок хитэ ан лавымердоп",имыртсо,"подремывал на этих коричневых, порезанных",острыми,,"ножиками сиденьях, в головах тоже",Андрей Волос. Недвижимость (2000) // «Новый Ми...,Андрей Волос,1955,Недвижимость,2000,художественная,роман,_,«Новый Мир»,2001,журнал,омонимия снята,"Электричка покачивалась, и, наверное, у всех, ..."
4,ёе илырку ино адог авд,еыртсо,два года они укрыли её,острые,,лопатки.,Людмила Улицкая. Казус Кукоцкого [Путешествие ...,Людмила Улицкая,1943,Казус Кукоцкого [Путешествие в седьмую сторону...,2000,художественная,роман,_,«Новый Мир»,2000,журнал,омонимия снята,Теперь Таня носила короткие чёрные платья и дл...


In [0]:
df_sharp_smell = pd.read_csv('ruscorpora_sharp_smell.csv')

In [10]:
df_sharp_smell.head()

,Reversed left context,Reversed center,Left context,Center,Punct,Right context,Title,Author,Birthday,Header,Created,Sphere,Type,Topic,Publication,Publ_year,Medium,Ambiguity,Full context
0,лыб инзиж оге в хапаз,йыртсо,запах в его жизни был,острый,,запах асидола ― жидкости для,Эдуард Лимонов. У нас была Великая Эпоха (1987),Эдуард Лимонов,1943,У нас была Великая Эпоха,1987,художественная,роман,_,NaN,NaN,книга,омонимия снята,Едва ли не первый запомнившийся сыну Вениамина...
1,с ыткудорП,мыртсо,Продукты с,острым,,"запахом ( сыр, рыба, копчёности,",Инструкция к холодильнику (1980),NaN,NaN,Инструкция к холодильнику,1980,"нехудожественная, производственно-техническая",инструкция,дом и домашнее хозяйство,NaN,NaN,прочая печатная продукция,омонимия снята,"Продукты сострым запахом ( сыр, рыба, копчёнос..."
2,";илокыв залг ьтох ,онмет олыб",йыртсо,"было темно, хоть глаз выколи;",острый,,"запах дыма свидетельствовал, что лучина",Д. В. Григорович. Бобыль (1847),Д. В. Григорович,1822,Бобыль,1847,художественная,рассказ,_,NaN,NaN,книга,омонимия снята,Совершенная тишина царствовала повсюду; в избе...
3,"и ,осям ястираж ецилу йокзу",йыртсо,"узкой улице жарится мясо, и",острый,,запах дразнит ноздри.,Дмитрий Шляпентох. Конец Истории: благословенн...,Дмитрий Шляпентох,1950,Конец Истории: благословенный Иов,2013,художественная,повесть,_,«Сибирские огни»,2013,журнал,омонимия не снята,"На узкой улице жарится мясо, иострый запах дра..."
4,"ее лашылс ,ееш йокосыв ан",йыртсо,"на высокой шее, слышал ее",острый,,запах.,"Роман Шмараков. Лошадь // «Сибирские огни», 2013",Роман Шмараков,1971,Лошадь,2013,художественная,рассказ,_,«Сибирские огни»,2013,журнал,омонимия не снята,"В полутьме я видел влажное движенье ее глаз, к..."


In [0]:
df_sharp_conflict = pd.read_csv('ruscorpora_sharp_conflict.csv')

In [12]:
df_sharp_conflict.head()

,Reversed left context,Reversed center,Left context,Center,Punct,Right context,Title,Author,Birthday,Header,Created,Sphere,Type,Topic,Publication,Publ_year,Medium,Ambiguity,Full context
0,лажлодорП,юуртсо,Продолжал,острую,,"борьбу против православной церкви и,",З. Н. Гиппиус. Задумчивый странник (о Розанове...,З. Н. Гиппиус,1869,Задумчивый странник (о Розанове),1923,"нехудожественная, публицистика",мемуары,_,NaN,NaN,книга,омонимия снята,Продолжалострую борьбу против православной цер...
1,огокакин йителитясед окьлоксен еишйажилб в,огортсо,в ближайшие несколько десятилетий никакого,острого,,конфликта не будет и быть,коллективный. «Китай грозит Америке войной». К...,коллективный,NaN,«Китай грозит Америке войной». Комментарии к с...,2015,"нехудожественная, электронная коммуникация",комментарии читателей,_,NaN,NaN,электронный текст,омонимия не снята,Так что никогда в ближайшие несколько десятиле...
2,теаджороп и атнетнокоедив яинелбертоп реткарах,еыртсо,характер потребления видеоконтента и порождает,острые,,споры об авторских правах и,Андрей Быстрицкий. Сериальная истерия // «Эксп...,Андрей Быстрицкий,NaN,Сериальная истерия,2014,"нехудожественная, публицистика",статья,"досуг, зрелища и развлечения, искусство и куль...",«Эксперт»,2014,журнал,омонимия не снята,Именно новый характер потребления видеоконтент...
3,к олевирп йинелватсдерп ыметсис йоксечиссалк,йортсо,классической системы представлений привело к,острой,,борьбе между различными подходами к,"Георгий Любарский. Мракобесие, или Наших бьют ...",Георгий Любарский,NaN,"Мракобесие, или Наших бьют",2013,"нехудожественная, публицистика",статья,"религия, социология",«Эксперт»,2013,журнал,омонимия не снята,Разрушение классической системы представлений ...
4,.оговиж мамроф и мадив едерс,еортсО,среде видам и формам живого.,Острое,,противоречие между двумя основополагающими те...,Георгий Малинецкий. От прошлого к будущему. Ро...,Георгий Малинецкий,1956,От прошлого к будущему. Российский контекст ме...,2013,"нехудожественная, публицистика",статья,"биология, математика, наука и технологии",«Знание-сила»,2013,журнал,омонимия не снята,среде видам и формам живого. Острое противореч...


# **AdaGram**

In [0]:
!wget https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib
clear_output()

In [0]:
morph = MorphAnalyzer()

In [0]:
tokenizer = RegexpTokenizer('[а-яА-ЯёЁ]+')

In [0]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [0]:
def lemmatize(text):
    return [morph.parse(word)[0].normal_form
            for word in tokenizer.tokenize(text) if word]

In [0]:
def adagram_disambiguate(df):
    contexts = [lemmatize(' '.join([str(df['Left context'][i]).strip(),
                                    df['Center'][i].strip(),
                                    str(df['Right context'][i]).strip()]))
                for i in range(len(df))]
    meanings = Counter([np.argmax(vm.disambiguate('острый', context))
                        for context in contexts])
    return meanings.most_common()[0][0]

In [19]:
adagram_disambiguate(df_sharp_objects)

3

В **AdaGram** значение "имеющий хорошо колющий конец или хорошо режущий край" имеет индекс 3.

In [20]:
vm.sense_neighbors('острый', 3)

[('заостренный', 1, 0.767315),
 ('заострять', 2, 0.73238033),
 ('зазубренный', 1, 0.6713583),
 ('коготь', 1, 0.6690367),
 ('зазубрина', 0, 0.6673185),
 ('тупой', 0, 0.6597572),
 ('изогнутый', 2, 0.6567419),
 ('зазубренный', 0, 0.6526937),
 ('раздваивать', 0, 0.64653206),
 ('лезвие', 1, 0.6378876)]

In [21]:
adagram_disambiguate(df_sharp_smell)

0

В **AdaGram** значение "сильно действующий на органы чувств" имеет индекс 0.

In [22]:
vm.sense_neighbors('острый', 0)

[('пряный', 0, 0.7627666),
 ('приправлять', 0, 0.7349547),
 ('приправа', 0, 0.724483),
 ('пикантный', 0, 0.72239685),
 ('перец', 1, 0.7065516),
 ('чесночный', 0, 0.7047403),
 ('кисло-сладкий', 0, 0.7045822),
 ('чили', 0, 0.7045208),
 ('карри', 0, 0.7006191),
 ('пряность', 0, 0.69331944)]

In [23]:
adagram_disambiguate(df_sharp_conflict)

2

В **AdaGram** значение "крайне напряженный; критический" имеет индекс 2.

In [24]:
vm.sense_neighbors('острый', 2)

[('злободневный', 0, 0.56665105),
 ('трудноразрешимый', 0, 0.54255724),
 ('обострять', 0, 0.5176394),
 ('насущный', 1, 0.51637393),
 ('назревать', 0, 0.5056267),
 ('конфликт', 0, 0.49893954),
 ('нерешенность', 0, 0.49728274),
 ('болезненный', 2, 0.47874802),
 ('нарастание', 1, 0.47848767),
 ('остро', 2, 0.4764235)]

## Припишем каждому словарному толкованию индекс значения **AdaGram**
*   **3** – Имеющий хорошо колющий конец или хорошо режущий край; противоп. тупой.
*   **0** – Сильно действующий на органы чувств (обоняние, вкус, слух, зрение); резкий.
*   **2** – Крайне напряженный; критический.

## Accuracy **AdaGram**
Посчитаем accuracy для модели **AdaGram**.

In [0]:
contexts_objects = [lemmatize(
    ' '.join([str(df_sharp_objects['Left context'][i]).strip(),
              df_sharp_objects['Center'][i].strip(),
              str(df_sharp_objects['Right context'][i]).strip()]))
    for i in range(len(df_sharp_objects))]
contexts_smell = [lemmatize(
    ' '.join([str(df_sharp_smell['Left context'][i]).strip(),
              df_sharp_smell['Center'][i].strip(),
              str(df_sharp_smell['Right context'][i]).strip()]))
    for i in range(len(df_sharp_smell))]
contexts_conflict = [lemmatize(
    ' '.join([str(df_sharp_conflict['Left context'][i]).strip(),
              df_sharp_conflict['Center'][i].strip(),
              str(df_sharp_conflict['Right context'][i]).strip()]))
    for i in range(len(df_sharp_conflict))]
contexts = contexts_objects + contexts_smell + contexts_conflict

При составлении *true_labels* будем предполагать, что в выборке из корпуса нет ошибок, и в каждом датасете содержатся только контексты одного значения.

In [0]:
true_labels = [3 for i in range(len(df_sharp_objects))] + \
              [0 for i in range(len(df_sharp_smell))] + \
              [2 for i in range(len(df_sharp_conflict))]
predicted_labels = [np.argmax(vm.disambiguate('острый', context))
                    for context in contexts]

In [27]:
print(f'Accuracy AdaGram: {accuracy_score(true_labels, predicted_labels)}')

Accuracy AdaGram: 0.7472959685349065


Достаточно неплохо.
# **WikiWordnet**
Придумаем способ соотносить толкования со значениями с помощью тезауруса (**WikiWordnet**). Для этого сначала вытащим из него все значения лексемы "острый".

In [0]:
wikiwordnet = WikiWordnet()

In [0]:
synsets = wikiwordnet.get_synsets('острый')

In [0]:
definitions = []

for synset in synsets:
    for word in synset.get_words():
        if word.lemma() == 'острый':
            definitions.append(word.definition())

In [31]:
pprint(definitions)

['острый~ru~острый~ru~<!-- 3 --> {{п.}} имеющий [[жгучий]] вкус, раздражающий '
 'слизистую оболочку рта {{пример|{{выдел|Острый}} перец.}}~48016~88302',
 'острый~ru~острый~ru~<!-- 8 --> [[критический]], [[катастрофичный]] '
 '{{пример|{{выдел|Острая}} нехватка средств при отсутствии доступных кредитов '
 'переросла в финансовый кризис.}}~15128~7782',
 'острый~ru~острый~ru~<!-- 1 --> способный легко [[резать]] или '
 '[[прокалывать]] {{пример|{{выдел|Острый}} нож.}} {{пример|{{выдел|Острая}} '
 'игла.|}}~1067~84838']


Значения из **WikiWordnet** очень удобно соотносятся с выбранными мной значениями. Запишем индексы **AdaGram** в список в том же порядке, что и соответствующие им значения в списке значений из **WikiWordnet**.

In [0]:
adagram_indices = [0, 2, 3]

Уберём всё ненужное (включая саму лексему "острый").

In [0]:
regex_to_sub = r'(({{пример\|{{выдел\|)?[Оо]стр(ый|ая)|(п\.))'

for i in range(len(definitions)):
    definitions[i] = re.sub(regex_to_sub, '', definitions[i])

In [34]:
pprint(definitions)

['~ru~~ru~<!-- 3 --> {{}} имеющий [[жгучий]] вкус, раздражающий слизистую '
 'оболочку рта }} перец.}}~48016~88302',
 '~ru~~ru~<!-- 8 --> [[критический]], [[катастрофичный]] }} нехватка средств '
 'при отсутствии доступных кредитов переросла в финансовый '
 'кризис.}}~15128~7782',
 '~ru~~ru~<!-- 1 --> способный легко [[резать]] или [[прокалывать]] }} нож.}} '
 '}} игла.|}}~1067~84838']


Векторизуем корпус контекстов и толкования с помощью **CountVectorizer** и **TfidfTransformer**, сохраним отдельно векторы толкований.

In [0]:
pipeline = Pipeline([('count', CountVectorizer()),
                     ('tfidf', TfidfTransformer())])

In [0]:
definitions_lemmas = [lemmatize(text) for text in definitions]

In [0]:
corpus = [' '.join(text) for text in (contexts + definitions_lemmas)]

In [0]:
definitions_vec = pipeline.fit_transform(corpus).toarray()[-3:]

Чтобы получить толкование для нового (уже лемматизированного) контекста, векторизуем этот контекст и найдём наиболее близкий (метрика **cosine similarity**) вектор толкования.

In [0]:
def wordnet_disambiguate(context):
    vect = pipeline.transform([' '.join(context)]).toarray()
    result_idx = np.argmax([cosine_similarity(vect, def_vect.reshape(1, -1))
                            for def_vect in definitions_vec])
    return definitions[result_idx], adagram_indices[result_idx]

# **Accuracy для WikiWordnet и AdaGram**
Посчитаем, в какой доле контекстов автоматически определённое с помощью **AdaGram** значение для контекста совпадает с автоматически определённым с помощью **WikiWordnet** значением толкования. *true_labels* – значения **WikiWordnet**, *predicted_labels* – значения **AdaGram**.

In [0]:
true_labels = [wordnet_disambiguate(context)[1] for context in contexts]
predicted_labels = [np.argmax(vm.disambiguate('острый', context))
                    for context in contexts]

In [41]:
print(f'Accuracy: {accuracy_score(true_labels, predicted_labels)}')

Accuracy: 0.2753195673549656


А вот сейчас accuracy значительно снизилась. Вероятно, **tf-idf** плохо подходит для автоматического определения значений толкований.